# ***End to End tutorial for SMS_SPAM labeling using JL:***
**The paper and documentation can be found here:** [Paper](https://arxiv.org/abs/2008.09887), [Documentation](https://spear-decile.readthedocs.io/en/latest/#joint-learning-jl)

In [1]:
#pip install

In [2]:
'''
User don't need to include this cell to use the package
'''
import sys
sys.path.append('../../') 

In [3]:
import numpy as np

# ***Defining an Enum to hold labels:***
### **Representation of class Labels**

<p>All the class labels for which we define labeling functions are encoded in enum and utilized in our next tasks. Make sure not to define an Abstain(Labeling function(LF) not deciding anything) class inside this Enum, instead import the ABSTAIN object as used later in LF section.</p>

<p>SPAM dataset contains 2 classes i.e <b>HAM</b> and <b>SPAM</b>. Note that the numbers we associate can be anything but it is suggested to use a continuous numbers from 0 to number_of_classes-1</p>

<p><b>**Note that even though this example is a binary classification, this(SPEAR) library supports multi-class classification**</b></p>

In [4]:
import enum

# enum to hold the class labels
class ClassLabels(enum.Enum):
    SPAM = 1
    HAM = 0

THRESHOLD = 0.8

# ***Defining preprocessors, continuous_scorers, labeling functions:***
During labeling the unlabelled data we lookup for few keywords to assign a class SMS.

<b>Example</b> : *If a message contains apply or buy in it then most probably the message is spam*

In [5]:
trigWord1 = {"free","credit","cheap","apply","buy","attention","shop","sex","soon","now","spam"}
trigWord2 = {"gift","click","new","online","discount","earn","miss","hesitate","exclusive","urgent"}
trigWord3 = {"cash","refund","insurance","money","guaranteed","save","win","teen","weight","hair"}
notFreeWords = {"toll","Toll","freely","call","meet","talk","feedback"}
notFreeSubstring = {"not free","you are","when","wen"}
firstAndSecondPersonWords = {"I","i","u","you","ur","your","our","we","us","youre"}
thirdPersonWords = {"He","he","She","she","they","They","Them","them","their","Their"}

### **Declaration of a simple preprocessor function**


For most of the tasks in NLP, computer vivsion instead of using the raw datapoint we preprocess the datapoint and then label it. Preprocessor functions are used to preprocess an instance before labeling it. We use **`@preprocessor(name,resources)`** decorator to declare a function as preprocessor.

In [6]:
from spear.labeling import preprocessor


@preprocessor(name = "LOWER_CASE")
def convert_to_lower(x):
    return x.lower().strip()

lower = convert_to_lower("RED")

### **Some Labeling function(LF) definitions**
Below are some examples on how to define LFs and continuous LFs(CLFs). To get the continuous score for a CLF, we need to define a function with continuous_scorer decorator(just like labeling_function decorator) and pass it to a CLF as displayed below. Also note how the continuous score can be used in CLF. Note that the word_similarity is the function with continuous_scorer decorator and is written in con_scorer file(this file is not a part of package) in same folder.

In [7]:
from spear.labeling import labeling_function, ABSTAIN

from helper.con_scorer import word_similarity
import re


@preprocessor()
def convert_to_lower(x):
    return x.lower().strip()


@labeling_function(resources=dict(keywords=trigWord1),pre=[convert_to_lower],label=ClassLabels.SPAM)
def LF1(c,**kwargs):    
    if len(kwargs["keywords"].intersection(c.split())) > 0:
        return ClassLabels.SPAM
    else:
        return ABSTAIN

@labeling_function(resources=dict(keywords=trigWord2),pre=[convert_to_lower],label=ClassLabels.SPAM)
def LF2(c,**kwargs):
    if len(kwargs["keywords"].intersection(c.split())) > 0:
        return ClassLabels.SPAM
    else:
        return ABSTAIN

@labeling_function(resources=dict(keywords=trigWord3),pre=[convert_to_lower],label=ClassLabels.SPAM)
def LF3(c,**kwargs):
    if len(kwargs["keywords"].intersection(c.split())) > 0:
        return ClassLabels.SPAM 
    else:
        return ABSTAIN

@labeling_function(resources=dict(keywords=notFreeWords),pre=[convert_to_lower],label=ClassLabels.HAM)
def LF4(c,**kwargs):
    if "free" in c.split() and len(kwargs["keywords"].intersection(c.split()))>0:
        return ClassLabels.HAM
    else:
        return ABSTAIN

@labeling_function(resources=dict(keywords=notFreeSubstring),pre=[convert_to_lower],label=ClassLabels.HAM)
def LF5(c,**kwargs):
    for pattern in kwargs["keywords"]:    
        if "free" in c.split() and re.search(pattern,c, flags= re.I):
            return ClassLabels.HAM
    return ABSTAIN

@labeling_function(resources=dict(keywords=firstAndSecondPersonWords),pre=[convert_to_lower],label=ClassLabels.HAM)
def LF6(c,**kwargs):
    if "free" in c.split() and len(kwargs["keywords"].intersection(c.split()))>0:
        return ClassLabels.HAM
    else:
        return ABSTAIN


@labeling_function(resources=dict(keywords=thirdPersonWords),pre=[convert_to_lower],label=ClassLabels.HAM)
def LF7(c,**kwargs):
    if "free" in c.split() and len(kwargs["keywords"].intersection(c.split()))>0:
        return ClassLabels.HAM
    else:
        return ABSTAIN

@labeling_function(label=ClassLabels.SPAM)
def LF8(c,**kwargs):
    if (sum(1 for ch in c if ch.isupper()) > 6):
        return ClassLabels.SPAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=trigWord1),pre=[convert_to_lower],label=ClassLabels.SPAM)
def CLF1(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.SPAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=trigWord2),pre=[convert_to_lower],label=ClassLabels.SPAM)
def CLF2(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.SPAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=trigWord3),pre=[convert_to_lower],label=ClassLabels.SPAM)
def CLF3(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.SPAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=notFreeWords),pre=[convert_to_lower],label=ClassLabels.HAM)
def CLF4(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.HAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=notFreeSubstring),pre=[convert_to_lower],label=ClassLabels.HAM)
def CLF5(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.HAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=firstAndSecondPersonWords),pre=[convert_to_lower],label=ClassLabels.HAM)
def CLF6(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.HAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=word_similarity,resources=dict(keywords=thirdPersonWords),pre=[convert_to_lower],label=ClassLabels.HAM)
def CLF7(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.HAM
    else:
        return ABSTAIN

@labeling_function(cont_scorer=lambda x: 1-np.exp(float(-(sum(1 for ch in x if ch.isupper()))/2)),label=ClassLabels.SPAM)
def CLF8(c,**kwargs):
    if kwargs["continuous_score"] >= THRESHOLD:
        return ClassLabels.SPAM
    else:
        return ABSTAIN

model loading
model loaded


# ***Accumulating all LFs into rules, an LFset(a class) object:***
### **Importing LFSet and passing LFs we defined, to that class**

In [8]:
from spear.labeling import LFSet

LFS = [LF1,
    LF2,
    LF3,
    LF4,
    LF5,
    LF6,
    LF7,
    LF8,
    CLF1,
    CLF2,
    CLF3,
    CLF4,
    CLF5,
    CLF6,
    CLF7,
    CLF8
      ]

rules = LFSet("SPAM_LF")
rules.add_lf_list(LFS)

# ***Loading data:***
### **Load the data: X, X_feats, Y**
<p>Note that the utils below is not a part of package but is used to load the necessary data. User have to use some means(which doesn't matter) to load his data(X, X_feats, Y). X is the raw data that is to be passed to LFs, X_feats is feature matrix, of type numpy array and shape (num_instances, num_features) and Y are true labels(if available). Note that we except user to provide feature matrix and feature matrix is not needed in Cage algorithm but it is needed in JL algorithm.</p>

In [9]:
from helper.utils import load_data_to_numpy, get_various_data

X, X_feats, Y = load_data_to_numpy()

validation_size = 100
test_size = 400
L_size = 100
U_size = 4500
n_lfs = len(rules.get_lfs())

X_V, Y_V, X_feats_V,_, X_T, Y_T, X_feats_T,_, X_L, Y_L, X_feats_L,_, X_U, X_feats_U,_ = get_various_data(X, Y,\
    X_feats, n_lfs, validation_size, test_size, L_size, U_size)

# ***Labeling data:***
### **Paths**
* path_json: path to json file generated by PreLabels
* V_path_pkl: path to pkl file generated by PreLabels containing the validation data with true labels
* L_path_pkl: path to pkl file generated by PreLabels containing the labeled data with true labels
* T_path_pkl: path to pkl file generated by PreLabels containing the test data with true labels
* U_path_pkl: path to pkl file generated by PreLabels containing the unlabelled data without true labels
* log_path: path to save the log which is generated during the algorithm
* params_path: path to save parameters of model

<p>Difference between test and labeled data is that labeled data may be used in the algorithm(JL uses it while Cage doesn't) but test data isn't. Make sure that the directory of the files(in above paths) exists. Note that any existing contents in pickle files will be erased.</p>

In [10]:
path_json = 'data_pipeline/JL/sms_json.json'
V_path_pkl = 'data_pipeline/JL/sms_pickle_V.pkl' #validation data - have true labels
T_path_pkl = 'data_pipeline/JL/sms_pickle_T.pkl' #test data - have true labels
L_path_pkl = 'data_pipeline/JL/sms_pickle_L.pkl' #Labeled data - have true labels
U_path_pkl = 'data_pipeline/JL/sms_pickle_U.pkl' #unlabelled data - don't have true labels

log_path_jl_1 = 'log/JL/sms_log_1.txt' #jl is an algorithm, can be found below
params_path = 'params/JL/sms_params.pkl' #file path to store parameters of JL, used below

### **Importing PreLabels class and using it to label data**
Json file should be generated only once as shown below.

In [11]:
from spear.labeling import PreLabels

sms_noisy_labels = PreLabels(name="sms",
                               data=X_V,
                               gold_labels=Y_V,
                               data_feats=X_feats_V,
                               rules=rules,
                               labels_enum=ClassLabels,
                               num_classes=2)
sms_noisy_labels.generate_pickle(V_path_pkl)
sms_noisy_labels.generate_json(path_json) #generating json files once is enough

sms_noisy_labels = PreLabels(name="sms",
                               data=X_T,
                               gold_labels=Y_T,
                               data_feats=X_feats_T,
                               rules=rules,
                               labels_enum=ClassLabels,
                               num_classes=2)
sms_noisy_labels.generate_pickle(T_path_pkl)

sms_noisy_labels = PreLabels(name="sms",
                               data=X_L,
                               gold_labels=Y_L,
                               data_feats=X_feats_L,
                               rules=rules,
                               labels_enum=ClassLabels,
                               num_classes=2)
sms_noisy_labels.generate_pickle(L_path_pkl)

sms_noisy_labels = PreLabels(name="sms",
                               data=X_U,
                               rules=rules,
                               data_feats=X_feats_U,
                               labels_enum=ClassLabels,
                               num_classes=2) #note that we don't pass gold_labels here, for the unlabelled data
sms_noisy_labels.generate_pickle(U_path_pkl)

100%|██████████| 4500/4500 [05:27<00:00, 13.75it/s]


# ***Accessing labeled data:***
### **Importing and the use of get_data and get_classes**
<p>These functions can be used to extract data from pickle files and json file respectively. Note that these are the files generated using PreLabels.</p>
<p>For detailed contents of output, please refer documentation.</p>

In [12]:
from spear.utils import get_data, get_classes

data_U = get_data(path = U_path_pkl, check_shapes=True)
#check_shapes being True(above), asserts for relative shapes of arrays in pickle file
print("Number of elements in data list: ", len(data_U))
print("Shape of feature matrix: ", data_U[0].shape)
print("Shape of labels matrix: ", data_U[1].shape)
print("Shape of continuous scores matrix : ", data_U[6].shape)
print("Total number of classes: ", data_U[9])

classes = get_classes(path = path_json)
print("Classes dictionary in json file(modified to have integer keys): ", classes)

Number of elements in data list:  10
Shape of feature matrix:  (4500, 1024)
Shape of labels matrix:  (4500, 16)
Shape of continuous scores matrix :  (4500, 16)
Total number of classes:  2
Classes dictionary in json file(modified to have integer keys):  {1: 'SPAM', 0: 'HAM'}


# ***Joint Learning(JL) Algorithm:***
## **Importing JL class (the algorithm) and declaring an object of it**
JL algoritm needs the four types of data:(all this data should be labeled using LFs via PreLabels class)
* Unlabeled data(doesn't have true/gold labels)
* labeled data(have true/gold labels)
* validation data(have true/gold labels)
* test data(have true/gold labels)

<p>All this data is compulsory for training(passed in fit_and_predict functions). Note that the amount of labeled or validation data can be small, for example they can be of the order of 100 each. Also refer subset selection to find the subset of the data, that is available with you, to label(using a trustable means/SMEs) and use it as 'labeled data' so that the data complements the LFs.</p>
<p>The member functions of JL can be choosen to return fm(feature model) or gm(graphical model) predictions. It is highly advised to use the predictions of fm.</p>
<p><b>Note:</b> Multiple calls to fit_* functions will train parameters continuously ie, parameters are not reinitialised in fit_* functions. So, to train large data, one can call fit_* functions repeatedly on smaller chunks. Also, in order to perform multiple runs over the algorithm, one need to reinitialise paramters(by creating an object of JL) at the start of each run.</p>

In [13]:
from spear.jl import JL

n_features = 1024
n_hidden = 512
feature_model = 'nn'
'''
'nn' is neural network. other alternative is 'lr'(logistic regression) which doesn't need n_hidden to be passed
during initialisation.
''' 

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)

### **fit_and_predict_proba function of JL class with two return values**
Here return_gm argument is True which returns predictions/probabilities from graphical model(Cage algorithm) along with feature model. Also note that here test data(path_T) is compulsory and metric_avg is just one value(instead of list as in Cage). The output(probs) is a numpy matrix of shape (num_instances, num_classes) having the probability of a particular instance being that class. 
<p> Here the order of classes along a row for any instance is the ascending order of values used in enum defined before to hold labels.</p>
<p>For more details about arguments, please refer documentation; same should be the case for any of the member functions used from here on.</p>

In [14]:
loss_func_mask = [1,1,1,1,1,1,1] 
'''
One can keep 0s in places where he don't want the specific loss function to be part
the final loss function used in training. Refer documentation(spear.JL.core.JL) to understand
the which index of loss_func_mask refers to what loss function.

Note: the loss_func_mask above may not be the optimal mask for sms dataset. We have to try
      some other masks too, to find the best one that gives good accuracies.
'''
batch_size = 150
lr_fm = 0.0005
lr_gm = 0.01
use_accuracy_score = False

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)

probs_fm, probs_gm = jl.fit_and_predict_proba(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, \
        path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = \
    lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = True, n_epochs = \
    100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary')

labels = np.argmax(probs_fm, 1)
print("probs_fm shape: ", probs_fm.shape)
print("probs_gm shape: ", probs_gm.shape)

 28%|██▊       | 28/100 [01:16<03:16,  2.72s/it]

early stopping at epoch: 28	best_epoch: 17
score used: f1_score
best_gm_val_score:0.4571428571428571	best_fm_val_score:0.6470588235294118
best_gm_test_score:0.56	best_fm_test_score:0.5263157894736842
best_gm_test_precision:0.4049586776859504	best_fm_test_precision:0.38461538461538464
best_gm_test_recall:0.9074074074074074	best_fm_test_recall:0.8333333333333334
probs_fm shape:  (4500, 2)
probs_gm shape:  (4500, 2)


### **fit_and_predict_proba function of JL class with one return value**
Here return_gm argument is False which returns predictions only from feature model. Also note the feature_model used here is 'lr'(logistic regression). The output(probs) is a numpy matrix of shape (num_instances, num_classes) having the probability of a particular instance being that class.
<p> Here the order of classes along a row for any instance is the ascending order of values used in enum defined before to hold labels.</p>

In [15]:
feature_model = 'lr' #in this case, n_hidden need not be passed as an argument

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model)

probs_fm = jl.fit_and_predict_proba(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, \
        path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = \
    lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = False, n_epochs = \
    100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary')

labels = np.argmax(probs_fm, 1)
print("probs_fm shape: ", probs_fm.shape)

 22%|██▏       | 22/100 [00:34<02:00,  1.55s/it]

early stopping at epoch: 22	best_epoch: 14
score used: f1_score
best_gm_val_score:0.4571428571428571	best_fm_val_score:0.8148148148148148
best_gm_test_score:0.5798816568047337	best_fm_test_score:0.9174311926605504
best_gm_test_precision:0.4260869565217391	best_fm_test_precision:0.9090909090909091
best_gm_test_recall:0.9074074074074074	best_fm_test_recall:0.9259259259259259
probs_fm shape:  (4500, 2)


### **fit_and_predict function of JL class**
The return_gm argument is set to True. The output(probs) is a numpy matrix of shape (num_instances,) containing integers(because need_strings is False), having the classes of each instance.

In [16]:
feature_model = 'nn' #resetting feature_model as 'nn'(neural network) for further trainings

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)

labels_fm, labels_gm = jl.fit_and_predict(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, \
        path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = \
    lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = True, n_epochs = \
    100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary', \
    need_strings = False)

print("labels_fm shape: ", labels_fm.shape)
print("labels_gm shape: ", labels_gm.shape)
print(type(labels_fm[0]))
print(type(labels_gm[0]))

 26%|██▌       | 26/100 [01:03<03:01,  2.45s/it]

early stopping at epoch: 26	best_epoch: 15
score used: f1_score
best_gm_val_score:0.4571428571428571	best_fm_val_score:0.7857142857142857
best_gm_test_score:0.5764705882352941	best_fm_test_score:0.7868852459016393
best_gm_test_precision:0.4224137931034483	best_fm_test_precision:0.7058823529411765
best_gm_test_recall:0.9074074074074074	best_fm_test_recall:0.8888888888888888
labels_fm shape:  (4500,)
labels_gm shape:  (4500,)
<class 'numpy.int64'>
<class 'numpy.int64'>


### **fit_and_predict function of JL class**
The return_gm argument is set to True. The output(probs) is a numpy matrix of shape (num_instances,) containing strings(because need_strings is True), having the classes of each instance.

In [17]:
jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)

labels_fm, labels_gm = jl.fit_and_predict(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, \
        path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = \
    lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = True, n_epochs = \
    100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary', \
    need_strings = True)

print("labels_fm shape: ", labels_fm.shape)
print("labels_gm shape: ", labels_gm.shape)
print(type(labels_fm[0]))
print(type(labels_gm[0]))

 31%|███       | 31/100 [01:23<03:06,  2.71s/it]

early stopping at epoch: 31	best_epoch: 20
score used: f1_score
best_gm_val_score:0.4571428571428571	best_fm_val_score:0.7333333333333334
best_gm_test_score:0.5568181818181819	best_fm_test_score:0.7692307692307692
best_gm_test_precision:0.4016393442622951	best_fm_test_precision:0.6578947368421053
best_gm_test_recall:0.9074074074074074	best_fm_test_recall:0.9259259259259259
labels_fm shape:  (4500,)
labels_gm shape:  (4500,)
<class 'numpy.str_'>
<class 'numpy.str_'>


### **Save parameters**
<p> Make sure that the directory of save_path file exists. Note that any existing contents in pickle file will be erased.</p>

In [18]:
jl.save_params(save_path = params_path)

### **Load parameters**

In [19]:
jl_2 = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)
jl_2.load_params(load_path = params_path)

### **predict_fm/gm_proba functions of JL class**
The output(probs_test) is a numpy matrix of shape (num_instances, num_classes) having the probability of a particular instance being that class.
<p> Here the order of classes along a row for any instance is the ascending order of values used in enum defined before to hold labels.</p>
<p>Note that predict_fm_proba takes feature matrix(can also be obtained from pickle file using get_data()) as argument while predict_gm_proba takes pickle file(containing labels given by LFs) as argument.</p>

In [20]:
probs_fm_test = jl_2.predict_fm_proba(x_test = X_feats_T)
probs_gm_test = jl_2.predict_gm_proba(path_test = T_path_pkl, qc = 0.85)

print("probs_fm_test shape: ", probs_fm_test.shape)
print("probs_gm_test shape: ", probs_gm_test.shape)

probs_fm_test shape:  (400, 2)
probs_gm_test shape:  (400, 2)


### **predict_fm/gm functions of JL class**
The output(probs) is a numpy matrix of shape (num_instances,) containing integers(strings) if need_strings is Flase(True), having the classes of each instance. Just the use case with need_strings as False is displayed here. 
<p>Note that predict_fm takes feature matrix(can also be obtained from pickle file using get_data()) as argument while predict_gm takes pickle file(containing labels given by LFs) as argument.</p>

In [21]:
labels_fm_test = jl_2.predict_fm(x_test = X_feats_T, need_strings=False)
labels_gm_test = jl_2.predict_gm(path_test = T_path_pkl, qc = 0.85, need_strings=False)

print("labels_fm_test shape: ", labels_fm_test.shape)
print("labels_gm_test shape: ", labels_gm_test.shape)

from sklearn.metrics import accuracy_score, f1_score

#Y_T is true labels of test data, type is numpy array of shape (num_instances,)
print("accuracy_score of gm: ", accuracy_score(Y_T, labels_gm_test), "| fm: ", accuracy_score(Y_T, labels_fm_test))
print("f1_score of gm: ", f1_score(Y_T, labels_gm_test, average = 'binary'), "| fm: ", f1_score(Y_T, labels_fm_test, average = 'binary'))

labels_fm_test shape:  (400,)
labels_gm_test shape:  (400,)
accuracy_score of gm:  0.805 | fm:  0.925
f1_score of gm:  0.5568181818181819 | fm:  0.7692307692307692


### **Converting numpy array of integers to enums**
The below utility from spear can help convert return values of predict_fm, predict_gm(obtained when need_strings is Flase) to a numpy array of enums

In [22]:
from spear.utils import get_enum

probs_fm_test_enum = get_enum(np_array = labels_fm_test, enm = ClassLabels) 
#the second argument is the Enum class defined at beginning
print(type(probs_fm_test_enum[0]))

<enum 'ClassLabels'>
